In [1]:
using TSSOS
using DynamicPolynomials
using CSV
using DataFrames
using Statistics

In [2]:
dis = CSV.read("distanceSet.csv", header=1);
taxi=[1.38,0.77,2.52,1.15,1.2,3.53,1.0,2.2,0.95,1.42,1.7,2.77,0.57, 0.81,21.47, 1.13,1.19, 0.83, 0.93, 3.3,0.71, 1.04, 11.2, 0.72, 2.63, 0.82, 10.32, 3.51,7.25, 0.48, 2.5, 1.5, 0.4, 1.86, 2.34, 1.42,2.6,1.4, 1.02, 5.96, 0.84, 1.2, 9.3, 1.66,2.0, 2.55, 2.04, 17.87,0.6, 0.57];

In [24]:
# new formulation 3 10/02/21
# calculate 3 objective: intra-fair inter-fair customer-careF3_tssos(5,5,taxi[1:5],0.5,0.5,0)
function F3_tssos_21(icount,jcount,taxi,lam1,lam2,lam3)
    #icount=5 # the number of customers
    #jcount=5 # the number of drivers
    #female=[2,4]
    #male=[1,3,5]
    female=collect(2:2:jcount)
    male=collect(1:2:jcount)
    
    mu1=10
    mu2=10
    mu3=0

    #lam1=1
    #lam2=1
    #lam3=0
    
    dis=rand(icount,jcount)
    
    # set nc operators
    @polyvar u[1:jcount] M[1:icount*jcount]; #z[1:jcount]; 
    var=vcat(u,M);

    #l2=[z[j]-1+(u[j]+0.01)/1.01 for j in 1:jcount];
    #l3=[z[j]+1-(u[j]+0.01)/1.01 for j in 1:jcount];
    #l4=[b1-(1/length(female))*sum(z[j] for j in female),b1-(1/length(male))*sum(z[j] for j in male)];
    #l5=[b2-sum(M[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount) for i in 1:icount];
    l6=[1-sum(M[(i-1)*(jcount)+j] for i in 1:icount) for j in 1:jcount];
    #l7=[M[(i-1)*(jcount)+j]*(M[(i-1)*(jcount)+j]-1) for i in 1:icount for j in 1:jcount]; #equality #numeq=25
    l8=[u[j] for j in 1:jcount];
    l9=[M[(i-1)*(jcount)+j] for i in 1:icount for j in 1:jcount];

    obj=lam1*sum((1-(u[j]+0.01)/1.01)^2 for j in 1:jcount)+lam2*(sum((u[j]+0.01)/1.01 for j in female)-sum((u[j]+0.01)/1.01 for j in male))^2+lam3*sum(M[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount for i in 1:icount)+sum(mu1*(sum(M[(i-1)*(jcount)+j]*(taxi[i]-dis[i,j]) for i in 1:icount)-u[j])^2  for j in 1:jcount)+sum(mu2*(sum(M[(i-1)*(jcount)+j] for j in 1:jcount)-1)^2  for i in 1:icount)+sum(M[(i-1)*(jcount)+j]*(M[(i-1)*(jcount)+j]-1) for i in 1:icount for j in 1:jcount)

    # pop
    pop=vcat(obj,l6); #,l7,l8,l9

    d=1; # the relaxation order
    opt,sol,data=cs_tssos_first(pop,var,d,numeq=icount*jcount,TS="MD",solution=true);
    #opt,sol,data=cs_tssos_higher!(data,TS="MD")
    
    u_sol=sol[1:jcount]
    M_sol=sol[jcount+1:end]
    #intra5=sum((1-(u_sol[j]+0.01)/1.01)^2 for j in 1:jcount)
    
    mu=sum((u_sol[j]+0.01)/1.01 for j in 1:jcount)/jcount
    muf=sum((u_sol[j]+0.01)/1.01 for j in female)/length(female)
    mum=sum((u_sol[j]+0.01)/1.01 for j in male)/length(male)
    
    intra4=1/(2*mu*jcount^2)*sum(abs((u_sol[k]+0.01)/1.01-(u_sol[j]+0.01)/1.01) for k in 1:jcount for j in 1:jcount)
    inter3=(sum((u_sol[j]+0.01)/1.01 for j in female)-sum((u_sol[j]+0.01)/1.01 for j in male))^2
    care=sum(M_sol[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount for i in 1:icount)
    
    intra2=abs(1/jcount*sum(((u_sol[j]+0.01)/(1.01*mu))*log((u_sol[j]+0.01)/(1.01*mu)) for j in 1:jcount))
    intra3=abs(1/jcount*sum(log((1.01*mu)/(u_sol[j]+0.01)) for j in 1:jcount))
    
    inter1=abs((length(female)/jcount)*muf/mu*log(muf/mu)+(length(male)/jcount)*mum/mu*log(mum/mu))
    inter2=abs((length(female)/jcount)*log(mu/muf)+(length(male)/jcount)*log(mu/mum))
    
    return [intra2,intra3,intra4,inter1,inter2,inter3,care]
    #return [intra4,inter3,care]

end

F3_tssos_21 (generic function with 1 method)

In [25]:
F3_tssos_21(5,5,taxi[1:5],0.5,0.5,0)

------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 6               
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve te

7-element Array{Float64,1}:
 0.012270639264374411
 0.012561025321331343
 0.0846632460531209
 0.008477827022527716
 0.008377593246846032
 0.10263704835006258
 2.553973090932176

In [26]:
# for pareto optimal
icount=5 # the number of customers
jcount=5 # the number of drivers
repeat=10
lam1_range=0:0.1:1  
#lam2_range=1:-0.1:0 #step 0.1
df_21 = DataFrame(Intra_Fair2=Float64[],Intra_Fair3=Float64[],Intra_Fair4=Float64[],Inter_Fair1=Float64[],Inter_Fair2=Float64[],Inter_Fair3=Float64[],Customer_Care=Float64[],lam1=Float64[],lam2=Float64[],lam3=Float64[])

for lam1 in lam1_range
    for lam2 in 0:0.1:(1-lam1)
        for r in 1:repeat
            obj=F3_tssos_21(icount,jcount,taxi[1:5],lam1,lam2,1-lam1-lam2)
            push!(df_21, push!(obj,lam1,lam2,1-lam1-lam2))
        end
    end
end

#df[!,:lam2] .= -df[!,:lam1].+1
#df[!,:lam3] .= 0

#CSV.write("ParetoOptimal.csv", df,header=true)

------------------------------------------------------
The clique sizes of varibles:
[16, 14, 9, 6]
[1, 2, 5, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 296             
  Cones                  : 0               
  Scalar variables       : 6               
  Matrix variables       : 57              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve term

3   7.5e+00  1.5e-01  2.5e-01  -4.82e-01  -8.277578883e+01  -8.489245713e+01  1.5e-01  0.05  
4   4.8e+00  9.6e-02  1.5e-01  5.04e-02   1.623022543e+01   1.438974490e+01   9.6e-02  0.06  
5   1.7e+00  3.3e-02  2.6e-02  3.94e-01   2.439967110e-01   -1.586416907e-01  3.3e-02  0.08  
6   7.8e-01  1.5e-02  5.4e-03  2.69e+00   -3.066642912e+00  -3.151140105e+00  1.5e-02  0.08  
7   5.7e-02  1.1e-03  7.2e-05  1.45e+00   -2.076204420e+00  -2.077976155e+00  1.1e-03  0.09  
8   1.8e-03  3.6e-05  4.1e-07  1.05e+00   -1.877168217e+00  -1.877220110e+00  3.6e-05  0.11  
9   2.2e-05  4.3e-07  5.3e-10  1.00e+00   -1.868098571e+00  -1.868099181e+00  4.3e-07  0.13  
10  9.2e-07  1.8e-08  4.5e-12  1.00e+00   -1.867989774e+00  -1.867989799e+00  1.8e-08  0.13  
11  6.5e-08  1.8e-09  8.5e-14  1.00e+00   -1.867986107e+00  -1.867986108e+00  1.3e-09  0.14  
Optimizer terminated. Time: 0.16    

optimum = -1.8679861065168941
------------------------------------------------------
The clique sizes of varibles:
[

2   1.6e+01  2.6e-01  4.2e-01  -7.52e-01  -6.966041349e+01  -7.150378007e+01  2.6e-01  0.08  
3   8.7e+00  1.4e-01  2.3e-01  -5.01e-01  -8.698930192e+01  -8.923199320e+01  1.4e-01  0.09  
4   5.7e+00  9.1e-02  1.4e-01  6.46e-02   1.527710410e+01   1.326555293e+01   9.1e-02  0.13  
5   2.0e+00  3.1e-02  1.9e-02  3.78e-01   4.843698984e+00   4.651070561e+00   3.1e-02  0.13  
6   7.3e-01  1.2e-02  2.8e-03  2.46e+00   -4.030175715e+00  -4.059632817e+00  1.2e-02  0.14  
7   1.1e-01  1.7e-03  1.6e-04  1.35e+00   -1.873098551e+00  -1.877910447e+00  1.7e-03  0.16  
8   2.1e-03  3.4e-05  4.3e-07  1.07e+00   -1.560757789e+00  -1.560848665e+00  3.4e-05  0.17  
9   1.5e-05  2.4e-07  2.6e-10  1.00e+00   -1.553461797e+00  -1.553462450e+00  2.4e-07  0.19  
10  6.3e-07  1.0e-08  2.2e-12  1.00e+00   -1.553413026e+00  -1.553413052e+00  9.9e-09  0.19  
11  2.5e-08  7.9e-10  1.7e-14  1.00e+00   -1.553411231e+00  -1.553411232e+00  4.0e-10  0.20  
Optimizer terminated. Time: 0.22    

optimum = -1.553411230

Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.02    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 296             
  Cones                  : 0               
  Scalar variables       : 6               
  Matrix variables       : 57              
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 296
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables       : 7                 conic  

7   5.9e-02  8.9e-04  5.3e-05  1.23e+00   -2.253943854e+00  -2.255554666e+00  8.9e-04  0.20  
8   3.3e-03  5.0e-05  6.6e-07  1.10e+00   -2.239677173e+00  -2.239747283e+00  5.0e-05  0.22  
9   2.0e-04  3.1e-06  1.0e-08  1.01e+00   -2.239945968e+00  -2.239950279e+00  3.1e-06  0.23  
10  1.4e-05  2.0e-07  1.7e-10  1.00e+00   -2.240015959e+00  -2.240016245e+00  2.0e-07  0.25  
11  8.6e-07  1.3e-08  2.7e-12  1.00e+00   -2.240021280e+00  -2.240021298e+00  1.3e-08  0.27  
12  7.3e-08  2.1e-09  6.7e-14  1.00e+00   -2.240021484e+00  -2.240021485e+00  1.1e-09  0.30  
Optimizer terminated. Time: 0.31    

optimum = -2.240021483901604
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366           

...

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 6               
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 366
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables       : 7                 conic                  : 7               
Optimizer  - Semi-definite variables: 52                scalarized             : 2129            
Factor     - setup time             : 0.01              dense det. time        : 0.00            
Factor     - ML order time          : 0.00              GP order time          : 0.00            
Factor     - nonzeros before factor :

1   3.2e+01  4.5e-01  6.2e-01  -8.88e-01  -2.725750427e+00  -3.695204785e+00  4.5e-01  0.13  
2   1.8e+01  2.6e-01  4.1e-01  -7.19e-01  -6.230835698e+01  -6.401080347e+01  2.6e-01  0.16  
3   8.6e+00  1.2e-01  1.9e-01  -4.14e-01  -7.035209963e+01  -7.219193911e+01  1.2e-01  0.19  
4   5.8e+00  8.2e-02  1.1e-01  3.76e-01   1.393004814e+01   1.246169987e+01   8.2e-02  0.22  
5   2.0e+00  2.8e-02  1.3e-02  7.01e-01   3.896435471e+00   3.796961698e+00   2.8e-02  0.23  
6   4.9e-01  7.0e-03  1.3e-03  1.98e+00   -3.403367034e+00  -3.422754194e+00  7.0e-03  0.27  
7   4.6e-02  6.5e-04  3.1e-05  1.23e+00   -2.658529873e+00  -2.659417502e+00  6.5e-04  0.30  
8   1.1e-03  1.5e-05  1.1e-07  1.05e+00   -2.623236588e+00  -2.623256340e+00  1.5e-05  0.31  
9   3.7e-05  5.2e-07  6.9e-10  1.00e+00   -2.622241130e+00  -2.622241808e+00  5.2e-07  0.33  
10  2.3e-06  3.2e-08  1.1e-11  1.00e+00   -2.622209667e+00  -2.622209709e+00  3.2e-08  0.34  
11  1.2e-07  2.1e-09  1.3e-13  1.00e+00   -2.622207837e+00  

------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 6               
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.03    
Problem
  Name                   :                 
  Objective sense        : max        

Excessive output truncated after 524307 bytes.

6   5.1e-01  8.4e-03  1.7e-03  1.99e+00   -3.436383497e+00  -3.455015856e+00  8.4e-03  0.13  
7   5.9e-02  9.6e-04  5.2e-05  1.27e+00   -2.672991548e+00  -2.673838560e+00  9.6e-04  0.14  
8   1.3e-03  2.2e-05  1.7e-07  1.08e+00   -2.638399069e+00  -2.638412407e+00  2.2e-05  0.16  


In [28]:
CSV.write("ParetoOptimal_21.csv", df_21,header=true)

"ParetoOptimal_21.csv"

# Old Formulation

In [36]:
F3_tssos_21(5,5,taxi[1:5],0.5,0.5,0)

------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 6               
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve te

7-element Array{Float64,1}:
 0.010421537623723588
 0.010371159305920988
 0.07875108426353188
 0.009604556491216623
 0.009487450682511084
 0.10312807654473781
 1.9073303474364536

In [8]:
# new formulation 3 19/11
# calculate 3 objective: intra-fair inter-fair customer-care
function F3_tssos(icount,jcount,taxi,lam1,lam2,lam3)
    #icount=5 # the number of customers
    #jcount=5 # the number of drivers
    #female=[2,4]
    #male=[1,3,5]
    female=collect(2:2:jcount)
    male=collect(1:2:jcount)
    
    mu1=10
    mu2=10
    mu3=0

    #lam1=1
    #lam2=1
    #lam3=0
    
    dis=rand(icount,jcount)
    
    # set nc operators
    @polyvar u[1:jcount] M[1:icount*jcount]; #z[1:jcount]; 
    var=vcat(u,M);

    #l2=[z[j]-1+(u[j]+0.01)/1.01 for j in 1:jcount];
    #l3=[z[j]+1-(u[j]+0.01)/1.01 for j in 1:jcount];
    #l4=[b1-(1/length(female))*sum(z[j] for j in female),b1-(1/length(male))*sum(z[j] for j in male)];
    #l5=[b2-sum(M[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount) for i in 1:icount];
    l6=[1-sum(M[(i-1)*(jcount)+j] for i in 1:icount) for j in 1:jcount];
    l7=[M[(i-1)*(jcount)+j]*(M[(i-1)*(jcount)+j]-1) for i in 1:icount for j in 1:jcount]; #equality #numeq=25

    obj=lam1*sum((1-(u[j]+0.01)/1.01)^2 for j in 1:jcount)+lam2*(sum((u[j]+0.01)/1.01 for j in female)-sum((u[j]+0.01)/1.01 for j in male))^2+lam3*sum(M[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount for i in 1:icount)+sum(mu1*(sum(M[(i-1)*(jcount)+j]*(taxi[i]-dis[i,j]) for i in 1:icount)-u[j])^2  for j in 1:jcount)+sum(mu2*(sum(M[(i-1)*(jcount)+j] for j in 1:jcount)-1)^2  for i in 1:icount)

    # pop
    pop=vcat(obj,l6,l7);

    d=1; # the relaxation order
    opt,sol,data=cs_tssos_first(pop,var,d,numeq=icount*jcount,TS="MD",solution=true);
    #opt,sol,data=cs_tssos_higher!(data,TS="MD")
    
    u_sol=sol[1:jcount]
    M_sol=sol[jcount+1:end]
    #intra5=sum((1-(u_sol[j]+0.01)/1.01)^2 for j in 1:jcount)
    
    mu=sum((u_sol[j]+0.01)/1.01 for j in 1:jcount)/jcount
    muf=sum((u_sol[j]+0.01)/1.01 for j in female)/length(female)
    mum=sum((u_sol[j]+0.01)/1.01 for j in male)/length(male)
    
    intra4=1/(2*mu*jcount^2)*sum(abs((u_sol[k]+0.01)/1.01-(u_sol[j]+0.01)/1.01) for k in 1:jcount for j in 1:jcount)
    inter3=(sum((u_sol[j]+0.01)/1.01 for j in female)-sum((u_sol[j]+0.01)/1.01 for j in male))^2
    care=sum(M_sol[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount for i in 1:icount)
    
    intra2=abs(1/jcount*sum(((u_sol[j]+0.01)/(1.01*mu))*log((u_sol[j]+0.01)/(1.01*mu)) for j in 1:jcount))
    intra3=abs(1/jcount*sum(log((1.01*mu)/(u_sol[j]+0.01)) for j in 1:jcount))
    
    inter1=abs((length(female)/jcount)*muf/mu*log(muf/mu)+(length(male)/jcount)*mum/mu*log(mum/mu))
    inter2=abs((length(female)/jcount)*log(mu/muf)+(length(male)/jcount)*log(mu/mum))
    
    return [intra2,intra3,intra4,inter1,inter2,inter3,care]
    #return [intra4,inter3,care]

end

LoadError: UndefVarError: df_run not defined

In [33]:
F3_tssos(5,5,taxi[1:5],0.5,0.5,0)

------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve te

7-element Array{Float64,1}:
 0.014244166433593611
 0.013803539392723739
 0.08636167013304498
 0.012512908004531934
 0.012351782298364561
 0.04063770364833207
 2.5180908064486256

In [34]:
# for pareto optimal
icount=5 # the number of customers
jcount=5 # the number of drivers
repeat=10
lam1_range=0:0.1:1  
#lam2_range=1:-0.1:0 #step 0.1
df = DataFrame(Intra_Fair2=Float64[],Intra_Fair3=Float64[],Intra_Fair4=Float64[],Inter_Fair1=Float64[],Inter_Fair2=Float64[],Inter_Fair3=Float64[],Customer_Care=Float64[],lam1=Float64[],lam2=Float64[],lam3=Float64[])

for lam1 in lam1_range
    for lam2 in 0:0.1:(1-lam1)
        for r in 1:repeat
            obj=F3_tssos(icount,jcount,taxi[1:5],lam1,lam2,1-lam1-lam2)
            push!(df, push!(obj,lam1,lam2,1-lam1-lam2))
        end
    end
end

#df[!,:lam2] .= -df[!,:lam1].+1
#df[!,:lam3] .= 0

#CSV.write("ParetoOptimal.csv", df,header=true)

------------------------------------------------------
The clique sizes of varibles:
[16, 14, 9, 6]
[1, 2, 5, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 296             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 57              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve term

5   2.3e+00  4.0e-02  2.6e-02  2.20e+00   -4.804706973e+01  -4.835370686e+01  4.0e-02  0.11  
6   4.2e-01  7.3e-03  1.6e-03  1.59e+00   -4.905767967e+00  -4.938222758e+00  7.3e-03  0.11  
7   8.4e-02  1.5e-03  1.4e-04  1.09e+00   4.507790247e-01   4.452649178e-01   1.5e-03  0.13  
8   2.6e-02  4.4e-04  2.1e-05  1.00e+00   8.208814886e-01   8.195767808e-01   4.4e-04  0.14  
9   6.8e-03  1.2e-04  2.8e-06  1.01e+00   9.752981361e-01   9.750056740e-01   1.2e-04  0.16  
10  1.3e-03  2.2e-05  2.2e-07  1.00e+00   9.940239078e-01   9.939748042e-01   2.2e-05  0.16  
11  4.5e-04  7.8e-06  4.4e-08  1.00e+00   1.000316949e+00   1.000300889e+00   7.8e-06  0.17  
12  1.7e-04  3.0e-06  1.1e-08  9.99e-01   1.002386046e+00   1.002379881e+00   3.0e-06  0.19  
13  3.2e-05  5.6e-07  8.3e-10  9.99e-01   1.003594402e+00   1.003593405e+00   5.6e-07  0.20  
14  5.5e-06  9.4e-08  5.9e-11  1.00e+00   1.003967076e+00   1.003966891e+00   9.4e-08  0.20  
15  4.2e-07  7.4e-09  1.3e-12  1.00e+00   1.004035021e+00   

optimum = 1.7098866990150885
------------------------------------------------------
The clique sizes of varibles:
[16, 14, 9, 6]
[1, 2, 5, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 296             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 57              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0

Factor     - ML order time          : 0.00              GP order time          : 0.00            
Factor     - nonzeros before factor : 2.17e+04          after factor           : 2.67e+04        
Factor     - dense dim.             : 0                 flops                  : 3.85e+06        
ITE PFEAS    DFEAS    GFEAS    PRSTATUS   POBJ              DOBJ              MU       TIME  
0   5.8e+01  1.0e+00  1.0e+00  0.00e+00   0.000000000e+00   0.000000000e+00   1.0e+00  0.03  
1   4.0e+01  6.9e-01  8.1e-01  -8.83e-01  -2.782754674e+00  -3.183702811e+00  6.9e-01  0.11  
2   3.2e+01  5.5e-01  7.1e-01  -9.27e-01  -4.157971232e+01  -4.229409429e+01  5.5e-01  0.14  
3   1.1e+01  1.9e-01  3.8e-01  -8.74e-01  -2.091878371e+02  -2.123521600e+02  1.9e-01  0.16  
4   4.3e+00  7.4e-02  9.0e-02  -3.35e-01  -2.102236033e+02  -2.113504327e+02  7.4e-02  0.19  
5   2.3e+00  3.9e-02  2.5e-02  2.19e+00   -4.613886701e+01  -4.642140344e+01  3.9e-02  0.22  
6   4.0e-01  6.8e-03  1.4e-03  1.58e+00   -4.410

4   5.0e+00  7.5e-02  9.1e-02  -3.34e-01  -2.063573200e+02  -2.074449827e+02  7.5e-02  0.13  
5   2.6e+00  3.9e-02  2.4e-02  2.16e+00   -4.511445081e+01  -4.537774893e+01  3.9e-02  0.14  
6   5.4e-01  8.1e-03  1.8e-03  1.58e+00   -5.437288443e+00  -5.468408894e+00  8.1e-03  0.16  
7   1.4e-01  2.1e-03  2.2e-04  1.10e+00   -3.790691331e-01  -3.859665139e-01  2.1e-03  0.19  
8   3.9e-02  5.8e-04  3.1e-05  1.03e+00   1.995359698e-01   1.979828526e-01   5.8e-04  0.20  
9   6.8e-03  1.0e-04  2.1e-06  1.02e+00   4.374581294e-01   4.372385435e-01   1.0e-04  0.22  
10  1.7e-03  2.6e-05  2.6e-07  9.93e-01   4.416922270e-01   4.416403546e-01   2.6e-05  0.25  
11  3.8e-04  5.8e-06  2.7e-08  9.99e-01   4.463560260e-01   4.463456388e-01   5.8e-06  0.27  
12  1.4e-04  2.1e-06  5.8e-09  9.99e-01   4.479603551e-01   4.479566524e-01   2.1e-06  0.28  
13  4.1e-05  6.2e-07  9.6e-10  1.00e+00   4.483789628e-01   4.483778214e-01   6.2e-07  0.30  
14  1.0e-05  1.5e-07  1.2e-10  1.00e+00   4.485448398e-01   

...

------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max        

4   3.9e+00  5.7e-02  3.8e-02  2.77e-03   -1.100970308e+02  -1.103040283e+02  5.7e-02  0.08  
5   1.0e+00  1.5e-02  3.9e-03  1.84e+00   -1.459888002e+01  -1.462793116e+01  1.5e-02  0.09  
6   2.0e-01  2.9e-03  3.2e-04  1.19e+00   -1.380775987e+00  -1.386190807e+00  2.9e-03  0.13  
7   4.6e-02  6.8e-04  3.7e-05  1.05e+00   7.371937948e-01   7.357851022e-01   6.8e-04  0.14  
8   1.4e-02  2.1e-04  6.4e-06  1.02e+00   9.070545919e-01   9.066116509e-01   2.1e-04  0.16  
9   3.7e-03  5.4e-05  8.3e-07  1.01e+00   9.847871247e-01   9.846698859e-01   5.4e-05  0.19  
10  8.7e-04  1.3e-05  9.4e-08  1.00e+00   1.000308827e+00   1.000281972e+00   1.3e-05  0.20  
11  1.7e-04  2.5e-06  8.2e-09  9.99e-01   1.005030577e+00   1.005025162e+00   2.5e-06  0.23  
12  3.7e-05  5.4e-07  8.2e-10  1.00e+00   1.005702597e+00   1.005701413e+00   5.4e-07  0.25  
13  3.6e-06  5.2e-08  2.4e-11  1.00e+00   1.005862055e+00   1.005861948e+00   5.2e-08  0.25  
14  1.8e-07  3.0e-09  2.8e-13  1.00e+00   1.005871071e+00   

optimum = 0.8974942002198315
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 :

Excessive output truncated after 524288 bytes.


------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               


In [35]:
CSV.write("ParetoOptimal_1.csv", df,header=true)

"ParetoOptimal_1.csv"